# Parse CellarTracker.com reviews

Parse the text file I have of amateur wine reviews from [CellarTracker.com](http://cellartracker.com). Note that this data file was downloaded from the web, and this is several years old. The website was scraped in a later notebook.

In [1]:
import pandas as pd
import numpy as np

import requests
import json
from bs4 import BeautifulSoup

import dill
import re
import time

Import the data. Note that it isn't in a csv format so it must be parsed.

In [2]:
! head -n 10 ../priv/csv/cellartracker.txt

wine/name: 1981 Ch&#226;teau de Beaucastel Ch&#226;teauneuf-du-Pape
wine/wineId: 18856
wine/variant: Red Rhone Blend
wine/year: 1981
review/points: 96
review/time: 1160179200
review/userId: 1
review/userName: Eric
review/text: Olive, horse sweat, dirty saddle, and smoke. This actually got quite a bit more spicy and expressive with significant aeration. This was a little dry on the palate first but filled out considerably in time, lovely, loaded with tapenade, leather, dry and powerful, very black olive, meaty. This improved considerably the longer it was open. A terrific bottle of 1981, 96+ and improving. This may well be my favorite vintage of Beau except for perhaps the 1990.



In [70]:
with open('../priv/csv/cellartracker.txt','r') as fh:
    data_str = fh.read()

In [71]:
data_list = re.split(r"""\n\n""", data_str)

## Parse reviews
Convert each entry to a Pandas Series and store it in a list.

In [81]:
series_list = list()

for dat in data_list:
    
    dat_list = [x.strip() for x in dat.split('\n') 
                if (x.startswith('wine') or x.startswith('review'))]
    
    series_list.append(pd.Series(dict([re.search(r"""((?:wine|review)\/.+?): (.+)""", 
                                       x.strip()).groups() for x in dat_list])).T)
    

In [82]:
data_df = pd.concat(series_list, axis=1).T

In [86]:
data_df = data_df.rename_axis(lambda x: x.replace('/', '_'), axis=1)

In [98]:
data_df['wine_name'] = data_df.wine_name.apply(lambda x: x.replace('&#226;','a'))
data_df['review_text'] = data_df.review_text.apply(lambda x: x.replace('&#226;','a'))
# data_df['review_points'] = data_df.review_points.replace('N/A', np.NaN)
data_df = data_df.replace('N/A',np.NaN)

In [99]:
data_df.head()

,review_points,review_text,review_time,review_userId,review_userName,wine_name,wine_variant,wine_wineId,wine_year
0,96,"Olive, horse sweat, dirty saddle, and smoke. T...",1160179200,1,Eric,1981 Chateau de Beaucastel Chateauneuf-du-Pape,Red Rhone Blend,18856,1981
1,93,A remarkably floral nose with violet and chamb...,1063929600,1,Eric,1995 Chateau Pichon-Longueville Baron,Red Bordeaux Blend,3495,1995
2,92,"Fantastic wine! Blackberry, smoke, olive, stem...",1195948800,1,Eric,2001 Thierry Allemand Cornas Reynard,Syrah,40451,2001
3,NaN,Perfect cork. Perfect fill. Somewhat alluring ...,1344124800,1,Eric,2001 Chateau Pavie Decesse,Red Bordeaux Blend,26767,2001
4,NaN,"OMFG, this wine just does not quit. I need to ...",1123718400,1,Eric,2003 Clos des Papes Chateauneuf-du-Pape,Red Rhone Blend,31665,2003


In [100]:
data_df.isnull().sum()

review_points      202271
review_text             1
review_time             0
review_userId           0
review_userName         0
wine_name               0
wine_variant            0
wine_wineId             0
wine_year           25926
dtype: int64

In [102]:
data_df.to_pickle('../priv/pkl/03_cellartracker_dot_com_data.pkl')